# 步骤
1. 函数封装
2. API:打包为Pip
3. Django:网页服务

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests, time, re, math, openpyxl, datetime, os, shutil, psutil
from tqdm import *
import xlwings as xw
from selenium import webdriver
#from splinter import Browser
#from pandas.io.json import json_normalize

# Def get position function: 增加程序对数据源结构变化的稳定性
def f_p(df, feature):
    feature_list = df.iloc[:,0].get_values().tolist() # 第一列为特征名称，Index为连续自然数
    return [i for i,x in enumerate(feature_list) if x == feature][0]

# Def Check Nan Function
# https://stackoverflow.com/questions/944700/how-can-i-check-for-nan-in-python
def is_nan(x):
    return isinstance(x, float) and math.isnan(x)

# Get current file path for migration consistency
path = os.getcwd().replace('\\','/') #r'%s' % os.getcwd().replace('\\','/')

# 定义函数：从文本中根据数字类型提取Int或者Float数值
def find_number(text):
    if len(re.findall(r'\.',text)) == 0:
        return float(re.findall(r'\d+',text.replace(",", ""))[0])
    else:
        return float(re.findall(r'\d+\.\d+',text.replace(",", ""))[0])

In [5]:
keywords = ['个股期权','期权','场外期权','衍生品','杠杆','配资']

In [6]:
df = pd.DataFrame(columns=['关键字','公众号名称','公众号网址','功能介绍','微信认证','最近文章'])

# type1:公众号搜索；type2:文章搜索
try:
    for keyword in tqdm(keywords):
        url = 'http://weixin.sogou.com/weixin?type=1&query='+keyword
        headers = {'User-Agent': str(UserAgent().random)}
        r = requests.get(url,headers)
        r.encoding = 'utf-8'
        soup = BeautifulSoup(r.text,'lxml')
        page_numbers = len(soup.find('div',class_='p-fy',id='pagebar_container').find_all('a'))

        for p in range(page_numbers):
            url = 'http://weixin.sogou.com/weixin?type=1&page='+str(p)+'&query='+'个股期权'
            r = requests.get(url,headers)
            r.encoding = 'utf-8'
            soup = BeautifulSoup(r.text,'lxml')

            for i in range(len(soup.find_all('li',id=True))):
                if len(soup.find_all('li',id=True)[i].find_all('dl')) == 1:
                    df = df.append({'关键字':keyword,\
                                    '公众号名称':soup.find_all('li',id=True)[i].find_all('a',target="_blank")[1].get_text(),\
                                    '功能介绍':soup.find_all('li',id=True)[i].find_all('dl')[0].get_text().replace('\n功能介绍：\n','').replace('\n','')\
                                   },ignore_index=True)
                elif len(soup.find_all('li',id=True)[i].find_all('dl')) == 2:
                    df = df.append({'关键字':keyword,\
                                    '公众号名称':soup.find_all('li',id=True)[i].find_all('a',target="_blank")[1].get_text(),\
                                    '功能介绍':soup.find_all('li',id=True)[i].find_all('dl')[0].get_text().replace('\n功能介绍：\n','').replace('\n',''),\
                                    '最近文章':soup.find_all('li',id=True)[i].find_all('dl')[1].get_text().split('\n\n')[1].split('document')[0]\
                                   },ignore_index=True)
                else:
                    df = df.append({'关键字':keyword,\
                                    '公众号名称':soup.find_all('li',id=True)[i].find_all('a',target="_blank")[1].get_text(),\
                                    '功能介绍':soup.find_all('li',id=True)[i].find_all('dl')[0].get_text().replace('\n功能介绍：\n','').replace('\n',''),\
                                    '微信认证':soup.find_all('li',id=True)[i].find_all('dl')[1].get_text().split('：\n')[1][:-1],\
                                    '最近文章':soup.find_all('li',id=True)[i].find_all('dl')[1].get_text().split('\n\n')[1].split('document')[0]\
                                   },ignore_index=True)
            time.sleep(5)
        time.sleep(5)
except AttributeError as e:
    print ('访问过于频繁，请设置time.sleep()!')

  0%|          | 0/6 [00:00<?, ?it/s]


访问过于频繁，请设置time.sleep()!


In [4]:
df

,关键字,公众号名称,公众号网址,功能介绍,微信认证,最近文章
0,个股期权,个股期权盈,NaN,"了解个股期权、场外期权从“个股期权盈”开始,股票期权的资讯与学习平台.",深圳高登投资咨询有限公司,
1,个股期权,个股期权咨询与询价,NaN,"个股期权、原油期货配资、商品投资咨询、指导,提供最新金融咨讯,资深分析师一对一解答",杭州恒牛金融信息服务有限公司,
2,个股期权,股票个股期权,NaN,"本公众号专注于场外个股期权相关业务,为客户提供及时的场外个股期权报价,价格优惠,服务到位!",NaN,"《五月财富倍增计划,神秘大咖,黄金策略》"
3,个股期权,个股期权操盘手,NaN,"场外个股期权行情,询价,开户,软件,下载.交易,投资,一对一服务,策略定制,专业操盘手,股票...",NaN,股票期权是什么?最标准的答案在这里
4,个股期权,个股期权大讲堂,NaN,"风靡美国的场外个股期权登陆中国,由国家颁发的正规出台法律,本公众平台力做最权威的个股期权讲解...",NaN,NaN
5,个股期权,个股期权6,NaN,个股期权的介绍和推广,NaN,NaN
6,个股期权,个股期权交易策略,NaN,"介绍交流个股期权交易规则、交易策略等,看好期权在中国市场的长远发展!!",NaN,NaN
7,个股期权,个股期权管家,NaN,"小金额撬动大资金,个股期权推荐,每天带你赚赚赚",NaN,NaN
8,个股期权,场外个股期权解说,NaN,暂时无提供任何产品,NaN,强势个股分析:002359北讯集团的周线走势图
9,个股期权,个股期权交易规则,NaN,投资A股就选 吾股哈哈 官网 www.wuguhaha.com 股票日内交易 杠杠比例放大1...,NaN,NaN


In [7]:
df.to_excel('期权公众号数据库.xlsx',encoding='gb18030')